In [1]:
import requests
import pandas as pd
import csv
import json, requests, re
from bs4 import BeautifulSoup as bs
from pprint import pprint
import datetime
import xml.etree.ElementTree as ET

## Downloaded Dataset


In [2]:
def data_parser():
    '''Data Source: Zillow Research
    Downloaded from: https://www.zillow.com/research/data/
        Home Values > 
            Data Type = ZHVI All Homes (SFR, Condo/Co-op) Time Series, Smoothed, Seasonally Adjusted($)
            Geography = ZIP Code
    Format: CSV
    Fields: ZIP code
    Attributes: home value index, size rank, city, county, metro, state, month
    '''

    with open("Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv", "r") as f:
        reader = csv.reader(f)
        reader_list = list(reader)
    reader_list_2 = []
    for item in reader_list:
        if not "" in item:
            reader_list_2.append(item)
    
    # Here we eliminate certain columns from our data and only include certain time periods
    # This is done through creating a new list that condenses the data from our previous list (reader_list_2)
    
    reader_list_3 = []
    for item in reader_list_2:
        reader_list_3.append([item[5], item[69], item[81], item[93], item[105], item[117], item[129], item[141], item[153], item[165], item[177], item[189], item[201], item[213], item[225], item[237], item[249]])
    
    reader_dict = {}
    for item in reader_list_3:
        if not item[0] == "State": # We do not need the header row so this is not included
            if not item[0] in reader_dict.keys():
                reader_dict[item[0]] = [[item[1], item[2], item[3], item[4], item[5], item[6], item[7], item[8], item[9], item[10], item[11], item[12], item[13], item[14], item[15], item[16]]]
            else:
                reader_dict[item[0]].append([item[1], item[2], item[3], item[4], item[5], item[6], item[7], item[8], item[9], item[10], item[11], item[12], item[13], item[14], item[15], item[16]])

    
    # We write our data to a separate file to document our progress
            
    with open('home_values_analysis.csv', "w") as f:
        dw = csv.DictWriter(f, fieldnames = ['State','2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'])
        dw.writeheader()


    # Our reader_dict is converted into a list (reader_list_4) that includes the data organized in the following format:
    # [state, [[chronological list of home values in state for region x], [chronological list of home values in state for region y], etc..]]
    

    reader_list_4 = []
    for (key, value) in reader_dict.items():
        reader_list_4.append([key, value])
    
    
    # The data in reader_dict_2 is organized as
    # {state: [2005_home_value, 2006_home_value, 2007_home_value, ......, 2020_home_value]}
    # this is done by keeping a recurring average for each state and count of # of regions averaged in each state
    
    reader_dict_2 = {}
        
    with open('home_values_analysis.csv', 'a') as f:
        writer = csv.writer(f)
        for state in reader_list_4:
            val_05 = val_06 = val_07 = val_08 = val_09 = val_10 = val_11 = val_12 = val_13 = val_14 = val_15 = val_16 = val_17 = val_18 = val_19 = val_20 = 0
            count = 0
            for item in state[1]:
                count += 1
                val_05 += float(item[0])
                val_06 += float(item[1])
                val_07 += float(item[2])
                val_08 += float(item[3])
                val_09 += float(item[4])
                val_10 += float(item[5])
                val_11 += float(item[6])
                val_12 += float(item[7])
                val_13 += float(item[8])
                val_14 += float(item[9])
                val_15 += float(item[10])
                val_16 += float(item[11])
                val_17 += float(item[12])
                val_18 += float(item[13])
                val_19 += float(item[14])
                val_20 += float(item[15])
            reader_dict_2[state[0]] = [round(val_05/count, 3), round(val_06, 3), round(val_07, 3), round(val_08, 3), round(val_09, 4), round(val_10/count, 3), round(val_11/count, 3), round(val_12/count, 3), round(val_13/count, 3), round(val_14/count, 3), round(val_15/count, 3), round(val_16/count, 3), round(val_17/count, 3), round(val_18/count, 3), round(val_19/count, 3), round(val_20/count, 3)]
            writer.writerow([state[0], val_05/count, val_06/count, val_07/count, val_08/count, val_09/count, val_10/count, val_11/count, val_12/count, val_13/count, val_14/count, val_15/count, val_16/count, val_17/count, val_18/count, val_19/count, val_20/count])


    # The data from this dictionary (reader_dict_2) is converted to a pandas DataFrame
    df = pd.DataFrame(data = reader_dict_2.values(), columns = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"], index = reader_dict_2.keys())
    print(df)
    
    # The pandas DataFrame is saved to another excel file
    writer = pd.ExcelWriter("Home_values.xlsx")
    df.to_excel(writer, sheet_name = "Home Values")
    writer.save()

############ Function Call ############
data_parser()

          2005         2006         2007         2008         2009  \
NY  272979.085  428669722.0  441771175.0  440281807.0  416810866.0   
IL  233075.000  111865123.0  115418929.0  113047224.0  101271250.0   
TX  160900.793   98729840.0  103238021.0  105562415.0  101387928.0   
CA  555719.208  820848466.0  830040020.0  757825091.0  609052397.0   
TN  121382.026   55175306.0   58836784.0   59825280.0   58185200.0   
FL  224612.237  225549386.0  239950037.0  210279190.0  167514467.0   
DC  433596.182   11123325.0   11207951.0   11321072.0   10803059.0   
NC  150623.338   53940961.0   57850967.0   59811541.0   58077096.0   
GA  151105.136   68848094.0   71733028.0   72009390.0   66859162.0   
OK   84550.974   31032390.0   32527694.0   33360956.0   33359237.0   
NJ  391942.076  250629524.0  256770354.0  247885672.0  227224068.0   
WA  261644.602   89438490.0   99901904.0  101589646.0   90745696.0   
AZ  203709.713   75255224.0   78648461.0   73189508.0   57309155.0   
MD  317955.957  1580

In [4]:
def data_parser2():
    '''Data Source: Zillow Research'''
    zillow = pd.read_csv('Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
    zillow.drop(columns=['RegionID', 'RegionType', 'StateName'], inplace=True)
    zillow.rename(columns={'SizeRank': 'Size Rank', 'RegionName': 'ZIP', 'CountyName': 'County'}, inplace=True)

    zillow = zillow.melt(id_vars=['Size Rank', 'ZIP', 'State', 'City', 'Metro', 'County'], var_name='Date', value_name='ZHVI')

    zillow['Date'] = [zillow['Date'][i][:8] + '01' for i in range(len(zillow['Date']))]
    """
    Testing out errors
    1. return zillow['Date'] - saved as variable and don't have to run it everytime
    2. figure out which slicing retains all date information
    3. 1-9: m.dd.yyyy 10-12 mm.dd.yyyy - if else statement: if len = else 
    """
    zillow = zillow.astype({'Date': 'datetime64[ns]'})
    
    zillow = zillow[~(zillow['ZHVI'].isna())]
    
    zillow.to_csv("Home_values_depth.csv")

    print(zillow)


############ Function Call ############
data_parser2()

         Size Rank    ZIP State           City  \
0                0  10025    NY       New York   
1                1  60657    IL        Chicago   
2                2  10023    NY       New York   
3                3  77494    TX           Katy   
4                4  60614    IL        Chicago   
...            ...    ...   ...            ...   
7961017      34430  20052    DC     Washington   
7961018      34430  72630    AR   Diamond City   
7961019      34430  12345    OR  Central Point   
7961020      34430  66045    KS       Lawrence   
7961021      34430   1470    MA         Groton   

                                    Metro                County       Date  \
0             New York-Newark-Jersey City       New York County 2000-01-01   
1                Chicago-Naperville-Elgin           Cook County 2000-01-01   
2             New York-Newark-Jersey City       New York County 2000-01-01   
3        Houston-The Woodlands-Sugar Land         Harris County 2000-01-01   
4        

## Web Collection HTML/XML


In [4]:
def web_parser1():
  '''
  Bureau of Economic Analysis API
  User guide: https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf
  Format: XML or JSON
  Fields: gross domestic product (gdp), consumer spending, personal income,
          consumer savings, international transactions, price indices, employment,
          government spending and assets
  Attributes: industry, geography (multiple levels), time (variable periodicity)
  Notes: -
  Work Documentation:
  A. API Call Selection: API organized by datasets with parameters
    1. Dataset List - list of dataset descriptions and names
    2. Regional Dataset Parameters - list of parameters, dtype, and description
      for Regional dataset
    3. Regional Dataset LineCodes - list of values for linecode parameter
    4. Regional Dataset TableNames - list of values for tablename parameter
  B. Preprocessing
    1. Parsed XML to DataFrame
  '''
  #Dataset List
  #Rerun only as needed - calls API each time
  # datasets_xml = requests.get('https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=GETDATASETLIST&ResultFormat=xml').text
  # datasets_soup = bs(datasets_xml, 'lxml')
  # datasets = datasets_soup.find_all('dataset')

  # pprint({dset['datasetdescription']: dset['datasetname'] for dset in datasets})

  #Regional Dataset Parameters
  #Rerun only as needed - calls API each time
  # regional_param_xml = requests.get('https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=getparameterlist&datasetname=Regional&ResultFormat=XML').text
  # regional_param_soup = bs(regional_param_xml, 'lxml')
  # regional_params = regional_param_soup.find_all('parameter')

  # pprint({param['parametername']: (param['parameterdatatype'], param['parameterdescription']) for param in regional_params})

  #Regional Dataset LineCodes
  #Rerun only as needed - calls API each time
  # regional_linecode_xml = requests.get('https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=getparametervalues&datasetname=Regional&parametername=linecode&ResultFormat=XML').text
  # regional_linecode_soup = bs(regional_linecode_xml, 'lxml')
  # regional_linecodes = regional_linecode_soup.find_all('paramvalue')

  #(linecodes, tablenames descriptions)
  # pprint([(pval['key'], pval['desc']) for pval in regional_linecodes])
  # pprint([(pval['key'], pval['desc']) for pval in regional_linecodes if '[CAGDP2]' == pval['desc'][:8]])
  # pprint([(pval['key'], pval['desc']) for pval in regional_linecodes if '1' == pval['key']])

  #Regional Dataset TableNames
  #Rerun only as needed - calls API each time
  # regional_tablename_xml = requests.get('https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=getparametervalues&datasetname=Regional&parametername=tablename&ResultFormat=XML').text
  # regional_tablename_soup = bs(regional_tablename_xml, 'lxml')
  # regional_tablenames = regional_tablename_soup.find_all('paramvalue')

  #{tablename: description}
  # pprint({pval['key']: pval['desc'] for pval in regional_tablenames})

  geofips = 'COUNTY'
  linecode = '1'
  tablename = 'CAGDP2'
  call = f'https://apps.bea.gov/api/data/?&UserID=973C0B87-9C21-4559-BF3D-AB7584B1445C&method=getdata&datasetname=Regional&geofips=%s&linecode=%s&tablename=%s&Year=ALL&ResultFormat=XML' % (geofips, linecode, tablename)

  # call

  #Rerun only as needed - calls API each time
  call_xml = requests.get(call).text
  call_soup = bs(call_xml, 'lxml')

  # pprint(call_soup.prettify())

  gdp_county = pd.DataFrame([(data["geoname"], data["timeperiod"], data["datavalue"]) for data in call_soup.find_all("data")], columns=['County', 'Year', 'GDP (thousands)'])
  
  writer = pd.ExcelWriter("General_GDP_Region.xlsx")
  gdp_county.to_excel(writer, sheet_name = "Economic Indicators")
  writer.save()

  print(gdp_county)



############ Function Call ############
web_parser1()

            County  Year GDP (thousands)
0      Autauga, AL  2001         748,918
1      Autauga, AL  2002         781,937
2      Autauga, AL  2003         804,299
3      Autauga, AL  2004         946,964
4      Autauga, AL  2005         969,849
...            ...   ...             ...
59199   Weston, WY  2015         299,245
59200   Weston, WY  2016         254,092
59201   Weston, WY  2017         273,154
59202   Weston, WY  2018         318,654
59203   Weston, WY  2019         319,777

[59204 rows x 3 columns]


## Web Collection: JSON

In [5]:
def web_parser2():
     """ Data Source: Bureau of Labor Statistics API
     Format: JSON *not accessible by no-parameter HTTP GET or URL in search browser
     Fields: price indices, employment, openings, hours, pay, benefits, injury
     Attributes: seasonal adjustment, sector, occupation, location, good/commodity,
                 socioeconomic demographics, time (variable periodicity)
     Notes: Max 10 years per request
     Work Documentation:
     A. Series ID Selection: API organizes data by Series IDs that limit requests
          and are difficult to pair to their meanings.
          1. BLS Popular Series ID JSON - set of 25 most requested Series ID
          2. BLS Series ID Format HTML - parsed site describing Series ID
          3. BLS Popular Series HTML - parsed site listing name and Series ID
               *HTTP GET returns different HTML than site
          4. Manual Renaming - After not finding a pairing of name and Series ID,
             I copy-and-pasted from a link I wish I had found earlier.
     B. Preprocessing
          1. Unwrapped JSON to DataFrame for each Series ID
          2. Set index to months_ago
          3. Joined on months_ago
          4. Changed type, filled nulls, removed "M" from 'month', created datetime type
     """
     #BLS Series ID Format HTML
     # series_html = requests.get('https://www.bls.gov/help/hlpforma.htm').text
     # series_soup = bs(series_html, 'html.parser')

     # h3_list = series_soup.find_all("h3")
     # site_id = [h3["id"].upper() for h3 in h3_list]
     # name = [h3.text for h3 in h3_list]

     # name[:5]

     # pre_list = series_soup.find_all("pre")
     # pre_str = [pre.text for pre in pre_list]

     # pre_str[-5:]

     # series_id = [re.search("Series ID    (.*)\r", pre).group() if re.search("Series ID    (.*)\r", pre) != None else re.search("Series ID   \t(.*)\r", pre).group() for pre in pre_str]
     # series_id = [pre[14:-2] if pre[13] in [" ", "t"] else pre[13:-2] for pre in series_id]

     # series_id[:5]

     # series_id_formats = pd.DataFrame(zip(name, series_id, site_id), columns=["Survey", "Example Series ID", "Site ID"])
     # series_id_formats["Prefix"] = [s[:2] for s in series_id_formats["Example Series ID"]]

     # series_id_formats.tail()

     #BLS Popular Series ID JSON
     #Rerun only as needed - calls API each time
     # popular_resp = requests.get('https://api.bls.gov/publicAPI/v2/timeseries/popular')
     # popular_series = popular_resp.json()

     # pprint(popular_series)

     # popular_id = [popular_series["Results"]["series"][i]["seriesID"] for i in range(25)]
     # prefix = [id[:2] for id in popular_id]

     # popular_series_df = pd.DataFrame(zip(popular_id, prefix), columns=["Series ID", "Prefix"])
     # popular_series_df = pd.merge(popular_series_df, series_id_formats, how="left")
     # popular_series_df.drop_duplicates(["Series ID"], keep=False, inplace=True)
     # popular_series_df.drop(["Prefix", "Example Series ID", "Site ID"], 1, inplace=True)
     # popular_series_df["Seasonal"] = ["Unseasonal" if id[2] == "U" else "Seasonal" for id in popular_series_df["Series ID"]]

     # popular_series_df.head()

     #BLS Popular Series HTML
     # popular_html = requests.get('https://data.bls.gov/cgi-bin/surveymost?bls').text
     # popular_soup = BeautifulSoup(series_html, 'html.parser')
     # popular_soup.find_all("dd")
     # popular_soup.body()

     #Manual Renaming
     copy_and_paste = """Civilian Labor Force (Adjusted) - LNS11000000
     Civilian Employment (Adjusted) - LNS12000000
     Civilian Unemployment (Adjusted) - LNS13000000
     Unemployment Rate (Adjusted) - LNS14000000
     Total Nonfarm Employment (Adjusted) - CES0000000001
     Total Private Average Weekly Hours of All Employees (Adjusted) - CES0500000002
     Total Private Average Weekly Hours of Prod. and Nonsup. Employees (Adjusted) - CES0500000007
     Total Private Average Hourly Earnings of All Employees (Adjusted) - CES0500000003
     Total Private Average Hourly Earnings of Prod. and Nonsup. Employees (Adjusted) - CES0500000008
     Non-farm Business Productivity - PRS85006092
     Nonfarm Business Unit Labor Costs - PRS85006112
     Nonfarm Business Real Hourly Compensation - PRS85006152
     Private Nonfarm Business - MPU4910012
     CPI for All Urban Consumers (CPI-U) 1982-84=100 (Unadjusted) - CUUR0000SA0
     PPI Final Demand (Adjusted) - WPSFD4
     PPI Final Demand (Unadjusted) - WPUFD4
     PPI Final Demand less foods and energy (Unadjusted) - WPUFD49104
     PPI Finished Goods 1982=100 (Unadjusted) - WPUFD49207
     Imports - EIUIR
     Exports - EIUIQ
     Employment Cost Index (ECI) Civilian (Unadjusted) - CIU1010000000000A
     ECI Private (Unadjusted) - CIU2010000000000A
     ECI Private Wage and Salaries (Unadjusted) - CIU2020000000000A"""

     names = re.findall(r"(.*?) - ", copy_and_paste)
     ids = re.findall(r" - (.*?)\n", copy_and_paste)

     # {sid: sname for sid, sname in zip(ids, names)}

     #BLS Popular Series JSON
     #Rerun only as needed - calls API each time
     headers = {"Content-type": "application/json"}
     data = json.dumps({"seriesid": ids, "startyear": "2011", "endyear": "2021"})
     # data = json.dumps({"seriesid": popular_series_df['Series ID'],"startyear": "2011", "endyear": "2021"})
     p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
     bls = p.json()

     # pprint(bls)

     bls_data = pd.DataFrame(bls['Results']['series'])

     # bls_data.head()

     bls_df = pd.DataFrame(bls_data['data'][0])
     bls_df.drop(columns=['footnotes', 'periodName'], inplace=True)
     bls_df.rename(columns={'value': names[0], 'period': 'month'}, inplace=True)
     bls_df = bls_df.astype({'year': int, **{col: float for col in bls_df.columns if col not in ['year', 'month']}}, copy=True)
     bls_df.fillna(0.0, inplace=True)

     bls_df.insert(0, 'months_ago', [i for i in range(len(bls_df['month']))])
     bls_df.set_index(keys=['months_ago'], drop=True, inplace=True)

     for i in list(range(1, len(bls_data))):
          bls_i = pd.DataFrame(bls_data['data'][i])
          bls_df.insert(len(bls_df.columns), names[i], bls_i['value'])
     
     #Popular Series JSON
     # for i in [2] + list(range(4, len(bls_data))):
          #     bls_i = pd.DataFrame(bls_data['data'][i])
          #     bls_df.insert(len(bls_df.columns), bls_data['seriesID'][i], bls_i['value'])

     # bls_df.head()

     #docs https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
     bls_df['month'] = [int(bls_df['month'][i][1:]) for i in range(len(bls_df['month']))]
     bls_df.insert(2, 'day', 1)
     bls_df.insert(0, 'Date', pd.to_datetime(bls_df[['year', 'month', 'day']]))
     bls_df.drop(columns=['year', 'month', 'day'], inplace=True)

     writer = pd.ExcelWriter("Employment_Other_stats.xlsx")
     bls_df.to_excel(writer, sheet_name = "Statistics")
     writer.save()
     
     print(bls_df)



############ Function Call ############
web_parser2()

                 Date  Civilian Labor Force (Adjusted)  \
months_ago                                               
0          2020-12-01                         160567.0   
1          2020-11-01                         160536.0   
2          2020-10-01                         160718.0   
3          2020-09-01                         160078.0   
4          2020-08-01                         160818.0   
...               ...                              ...   
115        2011-05-01                         153479.0   
116        2011-04-01                         153543.0   
117        2011-03-01                         153376.0   
118        2011-02-01                         153214.0   
119        2011-01-01                         153263.0   

                Civilian Employment (Adjusted)  \
months_ago                                       
0                                       149830   
1                                       149809   
2                                       14966

## Web Collection: HTML/XML 2

In [6]:
def extra_source1():
    '''
      Bureau of Economic Analysis API
      User guide: https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf
      Format: XML or JSON
      Fields: gross domestic product (gdp), consumer spending, personal income,
          consumer savings, international transactions, price indices, employment,
          government spending and assets
      Attributes: industry, geography (multiple levels), time (variable periodicity)
      Notes: -
      Work Documentation:
      A. Preprocessing
        1. Parsed XML to DataFrame
    '''
    tree = ET.parse("Phase2GDPIndustry.xml")
    root = tree.getroot()
    main_frame = root.findall("Results")
    
    for item in main_frame:
        data = item.findall("Data")
    
    # defining our selected years through keys in a dictionary (values are set to empty lists for now)
    Data_dict = {}
    Data_dict["2005"] = []
    Data_dict["2006"] = []
    Data_dict["2007"] = []
    Data_dict["2008"] = []
    Data_dict["2009"] = []
    Data_dict["2010"] = []
    Data_dict["2011"] = []
    Data_dict["2012"] = []
    Data_dict["2013"] = []
    Data_dict["2014"] = []
    Data_dict["2015"] = []
    Data_dict["2016"] = []
    Data_dict["2017"] = []
    Data_dict["2018"] = []
    Data_dict["2019"] = []
    Data_dict["2020"] = []

    industry_list = []
    accept_year_list = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]


    # values of empty list are filled with data for 2005-2020 years
    for item in data:
        # excluded catergory of "General government" and "Government enterprises"
        if not (item.attrib["IndustryDescription"] == "General government" or item.attrib["IndustryDescription"] == "Government enterprises"):
            if item.attrib["Year"] in accept_year_list and item.attrib["Quarter"] == "I":
                Data_dict[item.attrib["Year"]].append(item.attrib["DataValue"])
                if item.attrib["IndustryDescription"] not in industry_list:
                    industry_list.append(item.attrib["IndustryDescription"])


    clean_industry_list = []            
    
    # regex to clean up titles of industries
    for item in industry_list:
        clean_industry_list.append(re.sub(r"<.*>", "", item))
        
    df = pd.DataFrame(Data_dict, columns=Data_dict.keys(), index = clean_industry_list)
    
    writer = pd.ExcelWriter("GDP_in_depth.xlsx")
    df.to_excel(writer, sheet_name = "GDP Analysis")
    writer.save()
    
    print(df)
    
    
############ Function Call ############
extra_source1()

                                                2005     2006     2007  \
Agriculture, forestry, fishing, and hunting    131.0    121.1    145.1   
Farms                                          107.5     94.2    116.4   
Forestry, fishing, and related activities       23.5     26.9     28.6   
Mining                                         196.4    267.9    282.4   
Oil and gas extraction                         133.0    180.9    180.8   
...                                              ...      ...      ...   
Other real estate                              279.1    234.5    297.2   
Private goods-producing industries            2624.9   2866.7   2943.4   
Professional and business services            1413.5   1523.0   1615.1   
Private services-producing industries         8458.9   8973.5   9417.8   
Private industries                           11083.8  11840.2  12361.2   

                                                2008     2009     2010  \
Agriculture, forestry, fishing, and h